In [1]:
import json
import re

filtered_gen = json.load(open("notebooks/ravel_llama-3-8b_city_prompt_to_output.json", "r"))

In [3]:
from transformers import LlamaForCausalLM, PreTrainedTokenizerFast, LlamaConfig, LlamaModel
import torch

torch.set_default_device("cuda")
from torch import compile
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import transformers
from tqdm import tqdm
import pandas as pd
import yaml
import contextlib
import os
import time
import sys

sys.path.append("..")


%load_ext autoreload
%autoreload 2

from transformers import LlamaForCausalLM, PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("/work/frink/models/llama3-8B-HF")
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained("/work/frink/models/llama3-8B-HF")
for param in model.parameters():
    param.requires_grad = False
    
model.config.pad_token_id = model.config.eos_token_id
model.generation_config.pad_token_id = model.config.eos_token_id

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
statistics_dict = dict()

entity_pool = json.load(open("./data/ravel/ravel_city_entity_attributes.json", "r"))
prompt_pool = json.load(open("./data/ravel/ravel_city_attribute_to_prompts.json", "r"))

for entity in entity_pool.keys():
    statistics_dict[entity] = {}
    for attribute in entity_pool[entity].keys():
        statistics_dict[entity][attribute] = {i: False for i in range(len(prompt_pool[attribute]))}

for input_text, output_text in tqdm(filtered_gen.items()):    
    for attribute in prompt_pool.keys():
        for i, prompt in enumerate(prompt_pool[attribute]):
            
            for c in ["{", "}", "[", "]", "(", ")"]:
                prompt = prompt.replace(c, f"\{c}")
                input_text = input_text.replace(c, f"\{c}")
            
            reg_prompt = (prompt % "[A-Za-z]*")
            
            if re.match(reg_prompt, input_text):
                entity = re.search(reg_prompt.replace("[A-Za-z]*", "([A-Za-z]*)"), input_text).group(1)
                if entity in statistics_dict.keys():
                    statistics_dict[entity][attribute][i] = True       
        

100%|██████████| 969696/969696 [21:35<00:00, 748.31it/s]  


In [12]:
json.dump(statistics_dict, open("notebooks/ravel_llama-3-8b_city_prompt_to_output_statistics.json", "w"))

In [15]:
statistics_dict.keys()

dict_keys(['Aalborg', 'Aarau', 'Aarhus', 'Aba', 'Abadan', 'Abadla', 'Abai', 'Abakan', 'Abau', 'Abaza', 'Abeche', 'Abha', 'Abidjan', 'Abilene', 'Abohar', 'Abomey', 'Abuja', 'Abuna', 'Acarau', 'Acarigua', 'Acatlan', 'Accra', 'Achinsk', 'Acu', 'Adana', 'Adelaide', 'Aden', 'Adigrat', 'Adiyaman', 'Adrar', 'Agadez', 'Agadir', 'Agapa', 'Agartala', 'Agdam', 'Agen', 'Agordat', 'Agra', 'Agri', 'Ahar', 'Ahmedabad', 'Ahvaz', 'Aigua', 'Aiken', 'Aiquile', 'Ajmer', 'Akita', 'Akola', 'Akron', 'Aksu', 'Aksum', 'Aktau', 'Akure', 'Alatyr', 'Alausi', 'Albany', 'Albury', 'Aldama', 'Aldan', 'Aleg', 'Alenquer', 'Aleppo', 'Alert', 'Alesund', 'Alexandria', 'Aleysk', 'Algha', 'Algiers', 'Alicante', 'Alice', 'Allende', 'Allentown', 'Alliance', 'Almaty', 'Almenara', 'Almeria', 'Almirante', 'Alotau', 'Alpena', 'Alpine', 'Alta', 'Altamira', 'Altata', 'Altay', 'Altdorf', 'Alton', 'Altoona', 'Alvorada', 'Alwar', 'Amahai', 'Amapa', 'Amarillo', 'Amasya', 'Ambala', 'Ambanja', 'Ambato', 'Ambler', 'Ambon', 'Ambriz', 'Amde

In [16]:
statistics_dict["Aarau"]

{'Continent': {0: False,
  1: True,
  2: True,
  3: True,
  4: True,
  5: True,
  6: True,
  7: True,
  8: True,
  9: True,
  10: True,
  11: True,
  12: True,
  13: True,
  14: True,
  15: True,
  16: True,
  17: True,
  18: True,
  19: True,
  20: True,
  21: True,
  22: False,
  23: False,
  24: False,
  25: False,
  26: False,
  27: False,
  28: False,
  29: False,
  30: False,
  31: False,
  32: False,
  33: False,
  34: False,
  35: False,
  36: False,
  37: False,
  38: False,
  39: False,
  40: False,
  41: False,
  42: True},
 'Country': {0: True,
  1: True,
  2: False,
  3: False,
  4: True,
  5: True,
  6: True,
  7: False,
  8: False,
  9: False,
  10: False,
  11: False,
  12: False,
  13: False,
  14: False,
  15: False,
  16: False,
  17: False,
  18: False,
  19: False,
  20: False,
  21: False,
  22: False,
  23: False,
  24: False,
  25: False,
  26: False,
  27: True,
  28: True,
  29: True,
  30: True,
  31: True,
  32: True,
  33: True,
  34: True,
  35: True,
  36